In [1]:
import numpy as np
np.random.seed(123)
import tensorflow as tf
import keras
from functools import reduce
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers import Convolution2D, MaxPooling2D
# from keras.utils import np_utils
# from keras.datasets import mnist
from matplotlib import pyplot as plt

d:\00_data\pycharmprojects\opticjupyter\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\00_data\pycharmprojects\opticjupyter\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\00_data\pycharmprojects\opticjupyter\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\00_data\pycharmprojects\opticjupyter\venv\l

In [2]:
class yolo_eval_layer(keras.layers.Layer):
    def __init__(self, anchors, num_classes, image_shape, max_boxes=20, score_threshold=.6, iou_threshold=.5, **kwargs):
        self.boxes_ = []
        self.scores_ = []
        self.classes_ = []
        self.anchors = anchors
        self.num_classes = num_classes
        self.image_shape = image_shape
        self.max_boxes = max_boxes
        self.score_threshold = score_threshold
        self.iou_threshold = iou_threshold
        super(yolo_eval_layer, self).__init__(**kwargs)

    def build(self, input_shape):
        super(yolo_eval_layer, self).build(input_shape)

    def call(self, inputs, **kwargs):
        num_layers = len(inputs)
        anchor_mask = [[6, 7, 8], [3, 4, 5], [0, 1, 2]] if num_layers == 3 else [[3, 4, 5],
                                                                                 [1, 2, 3]]  # default setting
        input_shape = K.shape(inputs[0])[1:3] * 32
        boxes = []
        box_scores = []
        for l in range(num_layers):
            _boxes, _box_scores = yolo_boxes_and_scores(inputs[l],
                    self.anchors[anchor_mask[l]],  self.num_classes,  input_shape,  self.image_shape)
            boxes.append(_boxes)
            box_scores.append(_box_scores)
        boxes = K.concatenate(boxes, axis=0)
        box_scores = K.concatenate(box_scores, axis=0)

        mask = box_scores >= self.score_threshold
        max_boxes_tensor = K.constant(self.max_boxes, dtype='int32')

        boxes_ = []
        scores_ = []
        classes_ = []
        for c in range(self.num_classes):
            # TODO: use keras backend instead of tf.
            class_boxes = tf.boolean_mask(boxes, mask[:, c])
            class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
            nms_index = tf.image.non_max_suppression(
                class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=self.iou_threshold)
            class_boxes = K.gather(class_boxes, nms_index)
            class_box_scores = K.gather(class_box_scores, nms_index)
            classes = K.ones_like(class_box_scores, 'int32') * c
            boxes_.append(class_boxes)
            scores_.append(class_box_scores)
            classes_.append(classes)

        # self.boxes_ = K.concatenate(boxes_, axis=0)  #
        # self.scores_ = K.concatenate(scores_, axis=0)  #
        # self.classes_ = K.concatenate(classes_, axis=0)  #
        self.boxes_ = K.expand_dims(K.concatenate(boxes_, axis=0), axis=0)
        self.scores_ = K.expand_dims(K.concatenate(scores_, axis=0), axis=0)
        self.classes_ = K.expand_dims(K.concatenate(classes_, axis=0), axis=0)
        return [self.boxes_, self.scores_, self.classes_]

    def compute_output_shape(self, input_shape):
        return [K.shape(self.boxes_), K.shape(self.scores_), K.shape(self.classes_)]
        # [K.shape(2,), K.shape(1,), K.shape(1,)]
        # [K.shape(self.boxes_), K.shape(self.scores_), K.shape(self.classes_)]
        # [(self.num_classes, 4), (self.num_classes, 1), (self.num_classes, 1)]

def convolution_block(x, filters, rows, cols, padding='same', strides=(1, 1), bias=False):
    channel_axis = -1
    x1 = keras.layers.Convolution2D(filters, (rows, cols), strides=strides, padding=padding, use_bias=bias)(x)
#     tf.keras.layers.BatchNormalization()
    x2 = keras.layers.normalization.BatchNormalization()(x1)
    x3 = keras.layers.advanced_activations.ReLU()(x2)
    return x3

def merge_add_concat(intput_list, mode, concat_axis):
    if mode is 'concat':
        x = keras.layers.concatenate(intput_list, axis=concat_axis)
    else:
        x = keras.layers.Add()(intput_list)
    return x

def inception_stem(input_):
    channel_axis = -1

    # Input Shape is 299 x 299 x 3 (th) or 3 x 299 x 299 (th)
    x = convolution_block(input_, 32, 3, 3, strides=(2, 2), padding='valid')
    x = convolution_block(x, 32, 3, 3, padding='valid')
    x = convolution_block(x, 64, 3, 3)

    x1 = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)
    x2 = convolution_block(x, 96, 3, 3, strides=(2, 2), padding='valid')

    x = merge_add_concat([x1, x2], mode='concat', concat_axis=channel_axis)

    x1 = convolution_block(x, 64, 1, 1)
    x1 = convolution_block(x1, 96, 3, 3, padding='valid')

    x2 = convolution_block(x, 64, 1, 1)
    x2 = convolution_block(x2, 64, 1, 7)
    x2 = convolution_block(x2, 64, 7, 1)
    x2 = convolution_block(x2, 96, 3, 3, padding='valid')

    x = merge_add_concat([x1, x2], mode='concat', concat_axis=channel_axis)

    x1 = convolution_block(x, 192, 3, 3, strides=(2, 2), padding='valid')
    x2 = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)

    x = merge_add_concat([x1, x2], mode='concat', concat_axis=channel_axis)
    return x

def inception_a(input_):
    channel_axis = -1

    a1 = convolution_block(input_, 96, 1, 1)

    a2 = convolution_block(input_, 64, 1, 1)
    a2 = convolution_block(a2, 96, 3, 3)

    a3 = convolution_block(input_, 64, 1, 1)
    a3 = convolution_block(a3, 96, 3, 3)
    a3 = convolution_block(a3, 96, 3, 3)

    a4 = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input_)
    a4 = convolution_block(a4, 96, 1, 1)

    m = merge_add_concat([a1, a2, a3, a4], mode='concat', concat_axis=channel_axis)
    return m

def reduction_a(input_):
    channel_axis = -1

    r1 = convolution_block(input_, 384, 3, 3, strides=(2, 2), padding='valid')

    r2 = convolution_block(input_, 192, 1, 1)
    r2 = convolution_block(r2, 224, 3, 3)
    r2 = convolution_block(r2, 256, 3, 3, strides=(2, 2), padding='valid')

    r3 = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(input_)

    m = merge_add_concat([r1, r2, r3], mode='concat', concat_axis=channel_axis)
    return m

def inception_b(input_):
    channel_axis = -1

    b1 = convolution_block(input_, 384, 1, 1)

    b2 = convolution_block(input_, 192, 1, 1)
    b2 = convolution_block(b2, 224, 1, 7)
    b2 = convolution_block(b2, 256, 7, 1)

    b3 = convolution_block(input_, 192, 1, 1)
    b3 = convolution_block(b3, 192, 7, 1)
    b3 = convolution_block(b3, 224, 1, 7)
    b3 = convolution_block(b3, 224, 7, 1)
    b3 = convolution_block(b3, 256, 1, 7)

    b4 = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input_)
    b4 = convolution_block(b4, 128, 1, 1)

    m = merge_add_concat([b1, b2, b3, b4], mode='concat', concat_axis=channel_axis)
    return m

def reduction_b(input_):
    channel_axis = -1

    r1 = convolution_block(input_, 192, 1, 1)
    r1 = convolution_block(r1, 192, 3, 3, strides=(2, 2), padding='valid')

    r2 = convolution_block(input_, 256, 1, 1)
    r2 = convolution_block(r2, 256, 1, 7)
    r2 = convolution_block(r2, 320, 7, 1)
    r2 = convolution_block(r2, 320, 3, 3, strides=(2, 2), padding='valid')

    r3 = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(input_)

    m = merge_add_concat([r1, r2, r3], mode='concat', concat_axis=channel_axis)
    return m

def inception_c(input_):
    channel_axis = -1

    c1 = convolution_block(input_, 256, 1, 1)

    c2 = convolution_block(input_, 384, 1, 1)
    c2_1 = convolution_block(c2, 256, 1, 3)
    c2_2 = convolution_block(c2, 256, 3, 1)
    c2 = merge_add_concat([c2_1, c2_2], mode='concat', concat_axis=channel_axis)

    c3 = convolution_block(input_, 384, 1, 1)
    c3 = convolution_block(c3, 448, 3, 1)
    c3 = convolution_block(c3, 512, 1, 3)
    c3_1 = convolution_block(c3, 256, 1, 3)
    c3_2 = convolution_block(c3, 256, 3, 1)
    c3 = merge_add_concat([c3_1, c3_2], mode='concat', concat_axis=channel_axis)

    c4 = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input_)
    c4 = convolution_block(c4, 256, 1, 1)

    m = merge_add_concat([c1, c2, c3, c4], mode='concat', concat_axis=channel_axis)
    return m

def GlemConv2D(*args, **kwargs):
    """Wrapper to set Darknet parameters for Convolution2D."""
    glem_conv_kwargs = {'kernel_regularizer': keras.regularizers.l2(5e-4)}
    glem_conv_kwargs['padding'] = 'valid' if kwargs.get('strides')==(2,2) else 'same'
    glem_conv_kwargs.update(kwargs)
    return keras.layers.Conv2D(*args, **glem_conv_kwargs)

def GlemConv2D_BN_Leaky(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
    no_bias_kwargs = {'use_bias': False}
    no_bias_kwargs.update(kwargs)
    return compose(GlemConv2D(*args, **no_bias_kwargs), keras.layers.BatchNormalization(), keras.layers.LeakyReLU(alpha=0.1))

def resblock_body(x, num_filters, num_blocks):
    '''A series of resblocks starting with a downsampling Convolution2D'''
    # Darknet uses left and top padding instead of 'same' mode
    x = keras.layers.ZeroPadding2D(((1, 0), (1, 0)))(x)
    x = GlemConv2D_BN_Leaky(num_filters, (3, 3), strides=(2, 2))(x)
    for i in range(num_blocks):
        y = compose(GlemConv2D_BN_Leaky(num_filters//2, (1, 1)), GlemConv2D_BN_Leaky(num_filters, (3, 3)))(x)
        x = keras.layers.Add()([x,y])
    return x

def make_trunk_model():
    # make darknet53 trunk architecture
    inputs = Input(shape=(None, None, 3))
    x = GlemConv2D_BN_Leaky(32, (3, 3))(inputs)
    x = resblock_body(x, 64, 1)
    x = resblock_body(x, 128, 2)
    x = resblock_body(x, 256, 8)
    x = resblock_body(x, 512, 8)
    trunk_output = resblock_body(x, 1024, 4)
    return Model(inputs, trunk_output)

def make_last_layers(x, num_filters, out_filters):
    '''6 Conv2D_BN_Leaky layers followed by a Conv2D_linear layer'''
    x = compose(
        GlemConv2D_BN_Leaky(num_filters, (1,1)),
        GlemConv2D_BN_Leaky(num_filters*2, (3,3)),
        GlemConv2D_BN_Leaky(num_filters, (1,1)),
        GlemConv2D_BN_Leaky(num_filters*2, (3,3)),
        GlemConv2D_BN_Leaky(num_filters, (1,1)))(x)
    y = compose(GlemConv2D_BN_Leaky(num_filters*2, (3,3)), GlemConv2D(out_filters, (1,1)))(x)
    return x, y

def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.

    Reference: https://mathieularose.com/function-composition-in-python/
    """
    # return lambda x: reduce(lambda v, f: f(v), funcs, x)
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')



In [3]:
def create_inception_v4(input_shape, size=20, activation='softmax'):
    init = keras.layers.Input((input_shape[0], input_shape[1], 3))
    x = inception_stem(init)
    for i in range(4):
        x = inception_a(x)
    x = reduction_a(x)
    for i in range(7):
        x = inception_b(x)
    x = reduction_b(x)
    for i in range(3):
        x = inception_c(x)
#     before = x
#     based = Model(init, before)

    x = keras.layers.AveragePooling2D((8, 8))(x)

    x = keras.layers.Dropout(1.0)(x)
    x = keras.layers.Flatten()(x)

    new_out = keras.layers.Dense(size, activation=activation)(x)
    new_model = keras.models.Model(init, new_out, name='Inception-v4_fundus')
    return new_model #, based

def make_section2_model(trunk_model, anchors, class_names):
    num_anchors = len(anchors) // 3
    num_classes = len(class_names)

    x, branch2_output1 = make_last_layers(trunk_model.output, 512, num_anchors * (num_classes + 5))

    x = compose(GlemConv2D_BN_Leaky(256, (1, 1)), UpSampling2D(2))(x)
    x = keras.layers.Concatenate()([x, trunk_model.layers[152].output])
    x, branch2_output2 = make_last_layers(x, 256, num_anchors * (num_classes + 5))

    # 1*1 conv를 통해 channel 수를 줄이는 계산을 한다.
    x = compose(GlemConv2D_BN_Leaky(128, (1, 1)), UpSampling2D(2))(x)
    x = keras.layers.Concatenate()([x, trunk_model.layers[92].output])
    x, branch2_output3 = make_last_layers(x, 128, num_anchors * (num_classes + 5))

    branch2_outputs = [branch2_output1, branch2_output2, branch2_output3]

    model = keras.layers.Model(trunk_model.input, branch2_outputs)
    return model



In [4]:
# x1 = create_inception_v4((640, 640))
# x1.summary()

In [5]:
# x21 = make_section2_model(make_trunk_model(), self.anchors, self.section2_class_names)
# section2_out = yolo_eval_layer(self.anchors, len(self.section2_class_names), self.input_image_shape, score_threshold=self.score_threshold, iou_threshold=self.iou_threshold)(section2_model.output)
# x2 = Model([x21.input], outputs=[*section2_out])
# x2.summary()

In [6]:
# x3 = create_inception_v4((640, 640), size=64, activation="sigmoid")
# x3.summary()

In [7]:
# TEST

inputs = keras.layers.Input(shape=(None, None, 3))
x = GlemConv2D_BN_Leaky(32, (3, 3))(inputs)

Instructions for updating:
Colocations handled automatically by placer.
